# Prompt Engineering using CodeLlamma API


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# Assuming you're using CUDA, set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print(f'Available device: {device}')

Available device: cuda


# Load the dataset

In [4]:
from datasets import load_dataset

# Load ir1xor1
dataset = load_dataset("ASSERT-KTH/repairllama-datasets", "ir1xor1")
# Load irXxorY
# dataset = load_dataset("ASSERT-KTH/repairllama-dataset", "irXxorY")

def add_question(example):
    """ Add a new feature- question to the dataset """
    if 'question' not in example:
        example['question'] = 'What is the fix version of the code for the following vulnerability?'
    return example

def prepare_examples(dataset):
    """ Similarize the dataset by adding a question to the dataset  and renaming the columns"""
    dataset = dataset.map(add_question)
    # rename the columns
    dataset = dataset.rename_column('input', 'context')
    dataset = dataset.rename_column('output', 'answer')
    return dataset


dataset = prepare_examples(dataset)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 64643
    })
    test: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 1000
    })
})


In [6]:
# most lightweight model of CodeLlama for instruction prompt
model_id = "codellama/CodeLlama-7b-Instruct-hf"
# model_id = '/Users/guru/research/LLMs/CodeLlama-70-Instruct-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id, force_download=True)
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   # load_in_8bit=True,
   torch_dtype=torch.float16,
   device_map='auto',
)

/home/guru/miniconda3/envs/patch/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Tokenization

In [7]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""You are a powerful code-fixing model. Your job is to analyze and fix vulnerabilities in code. You are given a snippet of vulnerable code and its context.

You must output the fixed version of the code snippet.

### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
{data_point["answer"]}
"""
    return tokenize(full_prompt)

Reformat to prompt and tokenize each sample:

In [8]:
# tokenized_train_dataset = dataset.map(generate_and_tokenize_prompt)
dataset['test'].map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'answer', 'question', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

# Setup Lora

In [ ]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    # prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq


In [ ]:
model.train() # put model back into training mode
# model = prepare_model_for_int8_training(model)
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)


In [ ]:
def generate_chat_response(model, tokenizer, device, chat, max_new_tokens=200):

   inputs = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)
   output = model.generate(input_ids=inputs, max_new_tokens=max_new_tokens)
   output = output[0].to(device)
   return tokenizer.decode(output)


chat = [
   {"role": "system", "content": "You are a helpful and honest code assistant expert in JavaScript. Please, provide all answers to programming questions in JavaScript"},
   {"role": "user", "content": "Write a function that computes the set of sums of all contiguous sublists of a given list."},
]
response = generate_chat_response(model, tokenizer, device, chat)
print(response)


# RepairLLama

In [ ]:
from datasets import load_dataset

# Load ir1xor1
dataset = load_dataset("ASSERT-KTH/repairllama-datasets", "ir1xor1")
# Load irXxorY
# dataset = load_dataset("ASSERT-KTH/repairllama-dataset", "irXxorY")


In [ ]:
vul = dataset['test'][0]['input']
patch = dataset['test'][0]['output']

prompt = f"Fix the vulnerability in the following code:\n{vul}\n\nPatch:\n{patch}"


chat = [
   {"role": "system", "content": "You are a helpful and honest code assistant expert in Python. Please, provide all answers to programming questions in C"},
   {"role": "user", "content": prompt}]

response = generate_chat_response(model, tokenizer, device, chat, 200)
# print(response)

In [ ]:
# prompt = f"Fix the vulnerability in the following code:\n{vul}\n\nWhat is the patch?"


# chat = [
#    {"role": "system", "content": "You are a helpful and honest code assistant expert in Python. Please, provide all answers to programming questions in C"},
#    {"role": "user", "content": prompt}]

# response = generate_chat_response(model, tokenizer, device, chat, 200)
# print(response)

In [ ]:
# model.half()
# model.to(device)

# response = generate_chat_response(model, tokenizer, device, chat, 500)
# print(response)

In [ ]:
# # load the model
# model = AutoModelForCausalLM.from_pretrained('models/CodeLlama-7b-Instruct-hf')

# response = generate_chat_response(model, tokenizer, device, chat, 500)

# Fine-tuning CodeLLama model

In [ ]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

In [ ]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result


In [ ]:
dataset['train'][0]['input']

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
import torch
import gc


model.train() # put model back into training mode
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

# Preparing dataset for CodeLLama fine-tuning

In [9]:

from datasets import load_dataset
dataset_mc2 = load_dataset("b-mc2/sql-create-context", split="train")
train_dataset = dataset_mc2.train_test_split(test_size=0.1)["train"]
eval_dataset = dataset_mc2.train_test_split(test_size=0.1)["test"]
eval_dataset

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 7858
})

In [14]:
from datasets import load_dataset
# Load ir1xor1

def add_question(example):
    """ Add a new feature- question to the dataset """
    if "question" not in example:
        example[
            "question"
        ] = "What is the fix version of the code for the following vulnerability?"
    return example


def prepare_examples(dataset):
    """ Similarize the dataset by adding a question to the dataset  and renaming the columns"""
    dataset = dataset.map(add_question)
    # rename the columns
    dataset = dataset.rename_column("input", "context")
    dataset = dataset.rename_column("output", "answer")
    return dataset

dataset = load_dataset("ASSERT-KTH/repairllama-datasets", "ir1xor1")
dataset = prepare_examples(dataset)
train_data = dataset['train']
eval_data = dataset['test']
eval_data

Map:   0%|          | 0/64643 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'answer', 'question'],
    num_rows: 1000
})

In [18]:
tokenized_train_dataset = dataset['test'].map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [20]:
tokenized_train_dataset

Dataset({
    features: ['context', 'answer', 'question', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [29]:
def generate_eval_prompt(data_point):
    full_prompt = f"""You are a powerful code-fixing model. Your job is to analyze and fix vulnerabilities in code. You are given a snippet of vulnerable code and its context.

You must output the fixed version of the code snippet.

### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
"""
    return full_prompt

print(generate_eval_prompt(eval_data[0]))

You are a powerful code-fixing model. Your job is to analyze and fix vulnerabilities in code. You are given a snippet of vulnerable code and its context.

You must output the fixed version of the code snippet.

### Input:
What is the fix version of the code for the following vulnerability?

### Context:
	public synchronized void initComponentActivity(Body body) {
		if (!init) {
			this.logger = LoggerFactory.getLogger(this.getClass());
			logger.info("Initialising {} component.", this.getClass().getSimpleName());
			eleGenerator = new EleGeneratorForConstructQuery();
			try {
				String address = Constants.getProperties().getProperty("platfomservices.querydispatchapi.endpoint");
				soapServer = Endpoint.publish(address, this);
				logger.info("QueryDispatch SOAP service started at {}.", address);
			} catch (Exception e) {
				logger.error("Exception while publishing QueryDispatch SOAP Service", e);
			}
			try {
				restServer = new PlayPlatformservicesRest(this);
	        	logger.in

In [36]:
train_data.shuffle(seed=42).select(range(10))

Dataset({
    features: ['context', 'answer', 'question'],
    num_rows: 10
})

In [37]:
train_data.select(range(10))

Dataset({
    features: ['context', 'answer', 'question'],
    num_rows: 10
})